# Libraries

In [1]:
import os
import random
import time
import json
import warnings 
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from utils import *
import cv2

import numpy as np
import pandas as pd

# 전처리를 위한 라이브러리
from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as transforms

import albumentations as A
from albumentations.pytorch import ToTensorV2

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

plt.rcParams['axes.grid'] = False

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))

print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장

pytorch version: 1.4.0
GPU 사용 가능 여부: True
Tesla P40
1


# Hyperparamters And Seed

In [2]:
batch_size = 16   # Mini-batch size
num_epochs = 20
learning_rate = 0.0001

In [3]:
# seed 고정
random_seed = 21
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
# torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

# Configuration

In [ ]:
class Configuration:
    
    def __init__(self,
                 batch_size = 16,
                 num_epochs = 20,
                 learning_rate = 0.0001,
                 seed = 21,
                 val_every = 1,
                 num_workers = 4,
                 cutmix = False,
                 half = False,
                 train_resize = 224,
                 test_resize = 256,
                 encoder_name = 'senet154',
                 encoder_weights = "imagenet",
                 n_folds = 0, 
                 gkf = False, 
                 skf = False):
        
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.seed = seed
        self.val_every = val_every
        self.num_workers = num_workers
        self.cutmix = cutmix
        self.half = half
        self.train_resize = train_resize
        self.test_resize = test_resize
        self.encoder_name = encoder_name
        self.encoder_weights = encoder_weights
        self.n_folds = n_folds
        self.gkf = gkf
        self.skf = skf
        

In [ ]:
config = Configuration()

# CutMix

In [4]:
def rand_bbox(size, lam, half=False)->tuple:
    '''
    랜덤한 bounding box의 좌상단,우하단 좌표 반환

    Args:
        size (tuple): batch의 shape
        lam (float): 자를 비율
        half (bool): 절반으로 자름
    '''

    W = size[2]
    H = size[3]

    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    cx = np.random.randint(W)
    cy = np.random.randint(H)
    
    if half==False:
        bbx1 = np.clip(cx - cut_w // 2, 0, W) 
        bby1 = np.clip(cy - cut_h // 2, 0, H)
        bbx2 = np.clip(cx + cut_w // 2, 0, W)
        bby2 = np.clip(cy + cut_h // 2, 0, H)
    else:
        bbx1 = 0
        bby1 = 0
        bbx2 = W//2
        bby2 = H

    return bbx1, bby1, bbx2, bby2

def cutmix(image, mask, alpha, half=False):
    '''
    이미지와 마스크 컷믹스

    Args:
        image (tensor): batch 이미지
        mask (tensor): batch 마스크
        alpha (float): Beta Distribution의 alpha 값
    '''
  
    indices = torch.randperm(image.size(0)) # 배치 크기 입력

    lam = np.clip(np.random.beta(alpha, alpha),0.3,0.4)
    bbx1, bby1, bbx2, bby2 = rand_bbox(image.size(), lam, half)
    new_image = image.clone()
    new_mask = mask.clone()
    new_image[:, :, bby1:bby2, bbx1:bbx2] = image[indices, :, bby1:bby2, bbx1:bbx2]
    new_mask[:, bby1:bby2, bbx1:bbx2] = mask[indices, bby1:bby2, bbx1:bbx2]

    return new_image, new_mask

# Transform

In [5]:
train_transform = A.Compose([
                            A.Resize(224, 224),
                            ToTensorV2()
                            ])

val_transform = A.Compose([
                           A.Resize(224, 224),
                          ToTensorV2()
                          ])

test_transform = A.Compose([
                           A.Resize(256, 256),
                           ToTensorV2()
                           ])

# Collate Function

In [6]:
def collate_fn(batch):
    return tuple(zip(*batch))

# Dataset

In [44]:
category_names = ['Backgroud','UNKNOWN','General trash','Paper','Paper pack','Metal',
                    'Glass','Plastic','Styrofoam','Plastic bag','Battery','Clothing']
data_dir = '/opt/ml/input/data/train_all.json'

class TrashDataset(Dataset):
    """DataFrame format"""
    def __init__(self, dataframe, data_dir, mode = 'Train', transform = None):
        super().__init__()
        self.mode = mode
        self.transform = transform
        self.coco = COCO(data_dir)
        self.df = dataframe
        
    def __getitem__(self, index: int):
        
        image_id = self.df.iloc[index]['image_id']
        path = self.df.iloc[index]['path']
        width = self.df.iloc[index]['width']
        height = self.df.iloc[index]['height']      
        images = cv2.imread(path)
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.float32)
        images /= 255.0
        
        if self.mode == 'Train':
            bin = self.df.iloc[index]['bin']
            ann_ids = self.coco.getAnnIds(imgIds=image_id)
            anns = self.coco.loadAnns(ann_ids)

            cat_ids = self.coco.getCatIds()
            cats = self.coco.loadCats(cat_ids)

            masks = np.zeros((height,width))
            # Background = 0, Unknown = 1, General trash = 2, ... , Cigarette = 11
            for ann in anns:
                pixel_value = ann['category_id']+1
                masks = np.maximum(self.coco.annToMask(ann)*pixel_value, masks)
            masks = masks.astype(np.float32)

            if self.transform is not None:
                transformed = self.transform(image=images, mask=masks)
                images = transformed["image"]
                masks = transformed["mask"]
            
            return images, masks, bin
        
        if self.mode == 'Test':
            file_name = path.split('/')[-2:]
            file_name = "/".join(file_name)

            if self.transform is not None:
                transformed = self.transform(image=images)
                images = transformed["image"]
            
            return images, file_name
    
    
    def __len__(self) -> int:
        return len(self.df)

# DataLoader

## 전체 데이터셋에 대한 데이터로더

In [8]:
alldata = pd.read_csv('/opt/ml/code/alldata.csv')
all_dataset = TrashDataset(alldata, data_dir='/opt/ml/input/data/train_all.json', mode = 'Train', transform = train_transform)
all_dataloader = torch.utils.data.DataLoader(
                    all_dataset,
                    batch_size=batch_size,
                    pin_memory=False,
                    drop_last=False,
                    shuffle=True,        
                    num_workers=4,
                    collate_fn=collate_fn
                )

loading annotations into memory...
Done (t=4.10s)
creating index...
index created!


## K Fold를 위한 데이터로더

In [9]:
def get_train_valid_dataloader(df, trn_idx, val_idx,fold):

    from torch.utils.data import DataLoader
    
    data_dir = '/opt/ml/input/data/train_all.json'
    
    # 학습, 벨리데이션 데이터프레임 생성
    train_df = df.iloc[trn_idx,:].reset_index(drop=True)
    valid_df = df.iloc[val_idx,:].reset_index(drop=True)
    
    # 학습, 벨리데이션 데이터셋 생성
    print(f'\n###### Fold:{fold} - Loading Dataset ######\n')
    train_ds = TrashDataset(train_df, data_dir=data_dir, transform=train_transform, mode='Train')
    valid_ds = TrashDataset(valid_df, data_dir=data_dir, transform=val_transform, mode='Train')
    print(f'\n###### Fold:{fold} - Loading Dataset - DONE ######\n')
    
    # 학습, 벨리데이션 데이터로더 생성
    train_dataloader = DataLoader(
        train_ds,
        batch_size=batch_size,
        pin_memory=False,
        drop_last=False,
        shuffle=True,        
        num_workers=4,
        collate_fn=collate_fn,
    )
    val_dataloader = DataLoader(
        valid_ds, 
        batch_size=batch_size,
        num_workers=4,
        shuffle=False,
        pin_memory=False,
        collate_fn=collate_fn,
    )
    return train_dataloader, val_dataloader


## Test를 위한 데이터로더

In [28]:
def get_test_dataloader(df):
    data_dir = '/opt/ml/input/data/test.json'
    test_ds = TrashDataset(df,data_dir,'Test',test_transform)

    tst_dataloader = torch.utils.data.DataLoader(
                                                test_ds, 
                                                batch_size=batch_size,
                                                num_workers=4,
                                                shuffle=False,
                                                pin_memory=False,
                                                collate_fn=collate_fn,
                                                )
    
    return tst_dataloader

# Model

In [11]:
import segmentation_models_pytorch as smp
from pprint import pprint
dir(smp)

['DeepLabV3',
 'DeepLabV3Plus',
 'FPN',
 'Linknet',
 'PAN',
 'PSPNet',
 'Unet',
 'UnetPlusPlus',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'base',
 'deeplabv3',
 'encoders',
 'fpn',
 'linknet',
 'pan',
 'pspnet',
 'unet',
 'unetplusplus',
 'utils']

In [12]:
def get_model(encoder_name,encoder_weights,in_channels=3,classes=12):
    model = smp.DeepLabV3Plus(
    encoder_name=encoder_name,        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights=encoder_weights,     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=in_channels,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=classes,                      # model output channels (number of classes in your dataset)
)
    return model


In [13]:
encoder_name = "timm-regnetx_320"
encoder_weights = "imagenet"
model = get_model(encoder_name,encoder_weights)
model.eval();

In [14]:
from segmentation_models_pytorch.encoders import get_preprocessing_fn

preprocess_input = get_preprocessing_fn(encoder_name, pretrained=encoder_weights)

In [15]:
# 구현된 model에 임의의 input을 넣어 output이 잘 나오는지 test

#model = FCN8s(model = model, num_classes=12)
x = torch.randn([1, 3, 512, 512])
print("input shape : ", x.shape)
out = model(x).to(device)
print("output shape : ", out.size())

model = model.to(device)

input shape :  torch.Size([1, 3, 512, 512])
output shape :  torch.Size([1, 12, 512, 512])


# Train And Test

In [16]:
def validation(fold, epoch, model, valid_dataloader, criterion, device):
    print(f'\n- FOLD:{fold} VALIDATION #{epoch} START -\n')
    start_time = time.time()
    model.eval()
    hist = np.zeros((12, 12))
    with torch.no_grad():
        total_loss = 0
        cnt = 0
        for step, (images, masks, _) in enumerate(valid_dataloader):
            
            images = torch.stack(images).to(device)       # (batch, channel, height, width)
            masks = torch.stack(masks).long().to(device)  # (batch, channel, height, width)

            outputs = model(images)
            loss = criterion(outputs, masks)
            total_loss += loss
            cnt += 1
            
            outputs = torch.argmax(outputs, dim=1).detach().cpu().numpy()
            
            hist = add_hist(hist, masks.detach().cpu().numpy(), outputs, n_class=12)
            
        acc, acc_cls, mIoU, fwavacc = label_accuracy_score(hist)    
        avrg_loss = total_loss / cnt
        print(f'VALIDATION #{epoch}  Average Loss: {avrg_loss:.4f}, mIoU: {mIoU:.4f}, acc : {acc:.4f}')
    print(f'\n- FOLD:{fold} VALIDATION #{epoch} DONE - TIME:{time.time()-start_time}\n')
    return avrg_loss, mIoU

In [17]:
def train(fold, num_epochs, model, train_dataloader, valid_dataloader, criterion, optimizer, saved_dir, val_every, device, encoder_name):
    print(f'- Fold:{fold} Training Start- \n')
    start_time = time.time()
    best_loss = 9999999
    best_mIoU = 0
    for epoch in range(num_epochs):
        model.train()
        for step, (images, masks, bin) in enumerate(train_dataloader):
            images = torch.stack(images)       # (batch, channel, height, width)
            masks = torch.stack(masks).long()  # (batch, channel, height, width)
            
            # gpu 연산을 위해 device 할당
            images, masks = images.to(device), masks.to(device)

            #####################################
            # 50% 확률로 CutMix
            mix_decision = np.random.rand()
            if mix_decision < 0.5:
                # cutmix(data, target, alpha)
                images, masks = cutmix(images, masks, 1.)
            #####################################
                  
            # inference
            outputs = model(images).to(device)
            
            # loss 계산 (cross entropy loss)
            loss = criterion(outputs, masks)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # step 주기에 따른 loss 출력
            if (step + 1) % 25 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                    epoch+1, num_epochs, step+1, len(train_dataloader), loss.item()))
        
        # validation 주기에 따른 loss 출력 및 best model 저장
        if (epoch + 1) % val_every == 0:
            avrg_loss, mIoU = validation(fold, epoch + 1, model, valid_dataloader, criterion, device)
            if avrg_loss < best_loss:
                print('[loss] Best performance at epoch: {}'.format(epoch + 1))
                print('Save model in', saved_dir)
                best_loss = avrg_loss
                save_model(model, saved_dir, file_name = f'fold[{fold}]_loss_best_{encoder_name}(pretrained).pt')
            if mIoU > best_mIoU:
                print('[mIoU] Best performance at epoch: {}'.format(epoch + 1))
                print('Save model in', saved_dir)
                best_mIoU = mIoU
                save_model(model, saved_dir, file_name = f'fold[{fold}]_mIoU_best_{encoder_name}(pretrained).pt')
    
    print(f'\n- Fold:{fold} Training DONE - TIME:{time.time()-start_time}')

## 모델 저장 함수 정의

In [18]:
# 모델 저장 함수 정의
val_every = 1 
    
def save_model(model, saved_dir, file_name='fcn8s_best_model(pretrained).pt'):

    import os

    os.makedirs(saved_dir, exist_ok=True)

    # 모델 자체를 저장
    torch.save(model, saved_dir + '/'+ file_name)

## Group K Fold
- by = bin

In [19]:
def GKF(dataframe=pd.read_csv('/opt/ml/code/alldata.csv'),data_dir='/opt/ml/input/data/train_all.json',n_splits=5):

    from sklearn.model_selection import GroupKFold

    encoder_name = "senet154"
    encoder_weights = "imagenet"
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    saved_dir = '/opt/ml/code/saved/gkf'

    # bin 기준 나누기
    gkf = GroupKFold(n_splits=n_splits)
    folds = gkf.split(dataframe.values, y=None, groups=dataframe['bin'].values)

    for i, (trn_idx, val_idx) in enumerate(folds):
        # fold별 모델
        model = get_model(encoder_name,encoder_weights)
        model = model.to(device)
        
        # 데이터 로더
        train_dataloader, valid_dataloader = get_train_valid_dataloader(dataframe, trn_idx, val_idx,fold=i+1)

        # loss
        criterion = nn.CrossEntropyLoss()

        # optimizer
        optimizer = torch.optim.Adam(params = model.parameters(), lr = learning_rate, weight_decay=1e-6)

        # 학습
        train(fold=i+1,
              num_epochs=1, 
              model=model, 
              train_dataloader=train_dataloader, 
              valid_dataloader=valid_dataloader, 
              criterion=criterion, 
              optimizer=optimizer, 
              saved_dir=saved_dir, 
              val_every=val_every, 
              device=device, 
              encoder_name=encoder_name)


In [20]:
data_dir = '/opt/ml/input/data/train_all.json'
alldata = pd.read_csv('/opt/ml/code/alldata.csv')

GKF(alldata,data_dir,3)


###### Fold:1 - Loading Dataset ######
loading annotations into memory...
Done (t=4.12s)
creating index...
index created!
loading annotations into memory...
Done (t=4.25s)
creating index...
index created!
###### Fold:1 - Loading Dataset - DONE ######

- Fold:1 Training Start- 

Epoch [1/1], Step [25/153], Loss: 1.5079
Epoch [1/1], Step [50/153], Loss: 0.9618
Epoch [1/1], Step [75/153], Loss: 0.8725
Epoch [1/1], Step [100/153], Loss: 0.6802
Epoch [1/1], Step [125/153], Loss: 0.5313
Epoch [1/1], Step [150/153], Loss: 0.6674
Start validation #1
Validation #1  Average Loss: 0.4773, mIoU: 0.2210, acc : 0.8817
[loss] Best performance at epoch: 1
Save model in /opt/ml/code/saved/gkf
[mIoU] Best performance at epoch: 1
Save model in /opt/ml/code/saved/gkf

- Fold:1 Training DONE -

###### Fold:2 - Loading Dataset ######
loading annotations into memory...
Done (t=3.72s)
creating index...
index created!
loading annotations into memory...
Done (t=4.91s)
creating index...
index created!
###### Fo

## Stratified K Fold
- by = bin

In [ ]:
data_dir = '/opt/ml/input/data/train_all.json'

def SKF(dataframe=pd.read_csv('/opt/ml/code/alldata.csv'),data_dir='/opt/ml/input/data/train_all.json',n_splits=5):

    from sklearn.model_selection import StratifiedKFold

    encoder_name = "senet154"
    encoder_weights = "imagenet"
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    saved_dir = '/opt/ml/code/saved/skf'

    # bin 기준 나누기
    skf = StrarifiedKFold(n_splits=n_splits)
    folds = skf.split(dataframe.values, y=None, groups=dataframe['bin'].values)

    for i, (trn_idx, val_idx) in enumerate(folds):
        # fold별 모델
        model = get_model(encoder_name,encoder_weights)
        model = model.to(device)
        
        # 데이터 로더
        train_dataloader, valid_dataloader = get_train_valid_dataloader(dataframe, trn_idx, val_idx,fold=i+1)

        # loss
        criterion = nn.CrossEntropyLoss()

        # optimizer
        optimizer = torch.optim.Adam(params = model.parameters(), lr = learning_rate, weight_decay=1e-6)

        # 학습
        train(fold=i+1,
              num_epochs=1, 
              model=model, 
              train_dataloader=train_dataloader, 
              valid_dataloader=valid_dataloader, 
              criterion=criterion, 
              optimizer=optimizer, 
              saved_dir=saved_dir, 
              val_every=val_every, 
              device=device, 
              encoder_name=encoder_name)


In [ ]:
data_dir = '/opt/ml/input/data/train_all.json'
alldata = pd.read_csv('/opt/ml/code/alldata.csv')

SKF(alldata,data_dir,3)

# Load Model

In [ ]:
PATH = '/opt/ml/code/saved/gkf/fold[1]_loss_best_senet154(pretrained)/fold[1]_loss_best_senet154(pretrained).pt'
model = torch.load(PATH)

# Inference

In [70]:
def inference(folder_path):

    from glob import glob
    from scipy import stats

    model_list = glob(folder_path + '/*')
    test = pd.read_csv('/opt/ml/code/testdata.csv')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    size = 256

    n_folds = len(model_list)

    soft_voting = [] # Fold,Data,12,256,256
    hard_voting = [] # Fold,Data,256,256
    

    for fold in range(n_folds):

        print(f'\n@@@@@@@@@ FOLD {fold} INFERENCE START @@@@@@@@@ - TIME:0\n')
        start_time = time.time()
        
        file_name_list = []
        preds_array = np.empty((0, size*size), dtype=np.long)
        one_soft = [] # Data,12,256,256

        print(f'\n- FOLD {fold} DATASET LOAD START -\n')
        test_dataloader = get_test_dataloader(test)
        model = torch.load(model_list[fold])
        print(f'\n- FOLD {fold} DATASET LOAD DONE -\n')
        
        with torch.no_grad():
            for step, (imgs, image_infos) in enumerate(test_dataloader):
                
                if (step+1)%5==0 or step+1==len(test_dataloader):
                    print(f'STEP [{step+1}/{len(test_dataloader)}]')
                
                # inference (512 x 512)
                outs = model(torch.stack(imgs).to(device)) # batch, 12, 512, 512

                ######### soft voting #########
                soft = outs.detach().cpu().numpy()
                soft = soft.transpose(0,2,3,1)
                
                channel_list = []
                for image in soft:
                    transformed_mask = test_transform(image=image)['image']
                    channel_list.append(transformed_mask)

                # (batch, 12, 256, 256)
                soft = torch.stack(channel_list)
                soft = soft.numpy()
                
                one_soft.append(soft)
                ################################

                ######### hard voting #########
                oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()
                
                # resize (256 x 256)
                temp_mask = []
                for img, mask in zip(np.stack(imgs), oms):
                    transformed = A.Compose([A.Resize(size, size)])(image=img, mask=mask)
                    mask = transformed['mask']
                    temp_mask.append(mask)

                oms = np.array(temp_mask)
                
                oms = oms.reshape([oms.shape[0], size*size]).astype(int)
                preds_array = np.vstack((preds_array, oms))

                file_name_list.append([i for i in image_infos])
                
        # soft voting
        one_soft = np.array(one_soft)
        one_soft = np.concatenate(one_soft)
        soft_voting.append(one_soft)
        print(f'soft voting size: {one_soft.shape}')

        # hard voting
        hard_voting.append(preds_array)
        print(f'hard voting size: {preds_array.shape}')
        print(f'\n@@@@@@@@@ FOLD {fold} INFERENCE DONE @@@@@@@@@ - TIME:{time.time()-start_time}\n')
        
        # file names
        if fold == 1:
            file_names = [y for x in file_name_list for y in x]

        del model
        torch.cuda.empty_cache()
        
    return file_names, soft_voting, hard_voting

# submission.csv 생성

In [71]:
# test set에 대한 prediction
folder_path = '/opt/ml/code/saved/gkf'
encoder_name = 'senet154'
file_names, soft_voting, hard_voting = inference(folder_path)


@@@@@@@@@ FOLD 0 INFERENCE START @@@@@@@@@


- FOLD 0 DATASET LOAD START -

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

- FOLD 0 DATASET LOAD DONE -

STEP [5/53]
STEP [10/53]
STEP [15/53]
STEP [20/53]
STEP [25/53]
STEP [30/53]
STEP [35/53]
STEP [40/53]
STEP [45/53]
STEP [50/53]
STEP [53/53]
soft voting size: (837, 12, 256, 256)
hard voting size: (837, 65536)

@@@@@@@@@ FOLD 0 INFERENCE DONE @@@@@@@@@ - TIME:571.7046937942505


@@@@@@@@@ FOLD 1 INFERENCE START @@@@@@@@@


- FOLD 1 DATASET LOAD START -

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

- FOLD 1 DATASET LOAD DONE -

STEP [5/53]
STEP [10/53]
STEP [15/53]
STEP [20/53]
STEP [25/53]
STEP [30/53]
STEP [35/53]
STEP [40/53]
STEP [45/53]
STEP [50/53]
STEP [53/53]
soft voting size: (837, 12, 256, 256)
hard voting size: (837, 65536)

@@@@@@@@@ FOLD 1 INFERENCE DONE @@@@@@@@@ - TIME:572.0553107261658



# Post Process

In [ ]:
def ensemble_process(soft_voting,hard_voting):
    print(f'\n---------- ENSEMBLING START ---------- TIME:0')
    start_time = time.time()
    soft_voting = np.array(soft_voting) # Fold,Data,12,256,256
    soft_voting = np.sum(soft_voting, axis=0) # Data,12,256,256
    soft_voting = np.argmax(soft_voting, axis=1) # Data,256,256
    soft_voting = soft_voting.reshape([soft_voting.shape[0], size*size]).astype(int)# Data,256*256
    
    hard_voting = np.array(hard_voting) # Fold,Data,256*256
    hard_voting = stats.mode(hard_voting)[0] # Data,256*256
    hard_voting = np.squeeze(hard_voting)
    print(f'\n---------- ENSEMBLING DONE ---------- TIME:{time.time()-start_time}')
    return soft_voting, hard_voting

In [ ]:
soft_voting, hard_voting = ensemble_process(soft_voting, hard_voting)

# Soft Voting

In [73]:
from tqdm import tqdm

submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)

for file_name, string in tqdm(zip(file_names, soft_voting)):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)
submission.to_csv(f"/opt/ml/code/submission/{encoder_name}(pretrained)_soft_voting.csv", index=False)

837it [00:15, 54.88it/s]


In [79]:
softdata = pd.read_csv(f"/opt/ml/code/submission/{encoder_name}(pretrained)_soft_voting.csv")
softdata

,image_id,PredictionString
0,batch_01_vt/0021.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,batch_01_vt/0028.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,batch_01_vt/0031.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,batch_01_vt/0032.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,batch_01_vt/0070.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
...,...,...
832,batch_03/0947.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
833,batch_03/0968.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
834,batch_03/0969.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
835,batch_03/0992.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


# Hard Voting

In [87]:
from tqdm import tqdm

submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)
    
for file_name, string in tqdm(zip(file_names, hard_voting)):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

submission.to_csv(f"/opt/ml/code/submission/{encoder_name}(pretrained)_hard_voting.csv", index=False)

837it [00:15, 54.66it/s]


In [88]:
harddata = pd.read_csv(f"/opt/ml/code/submission/{encoder_name}(pretrained)_hard_voting.csv")
harddata

,image_id,PredictionString
0,batch_01_vt/0021.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,batch_01_vt/0028.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,batch_01_vt/0031.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,batch_01_vt/0032.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,batch_01_vt/0070.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
...,...,...
832,batch_03/0947.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
833,batch_03/0968.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
834,batch_03/0969.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
835,batch_03/0992.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
